In [4]:
from google.colab import files
files.upload()

{}

In [0]:
import numpy as np
import pandas as pd
import re
import nltk
import matplotlib.pyplot as plt

In [0]:
data_source = "Sentiment.csv"  
data = pd.read_csv(data_source) 

In [10]:
data.head()


,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,name,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,I_Am_Kenzi,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,8/7/2015 9:54,6.296970e+17,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,PeacefulQuest,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,8/7/2015 9:54,6.296970e+17,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,PussssyCroook,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,8/7/2015 9:54,6.296970e+17,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,MattFromTexas31,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,8/7/2015 9:54,6.296970e+17,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,sharonDay5,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,8/7/2015 9:54,6.296970e+17,NaN,Arizona


In [11]:
print (data.columns)

Index(['id', 'candidate', 'candidate_confidence', 'relevant_yn',
       'relevant_yn_confidence', 'sentiment', 'sentiment_confidence',
       'subject_matter', 'subject_matter_confidence', 'candidate_gold', 'name',
       'relevant_yn_gold', 'retweet_count', 'sentiment_gold',
       'subject_matter_gold', 'text', 'tweet_coord', 'tweet_created',
       'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')


In [12]:
print (data.shape)

(13871, 21)


In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
#features=le.fit_transform(data['text'])

#label1=le.fit_transform(data['sentiment'])
#label2=le.fit_transform(data['sentiment_confidence'])

#labels=list(zip(label1,label2))


features=data.iloc[:,15].values
labels=data.iloc[:,5].values

print(features)
print(labels)


['RT @NancyLeeGrahn: How did everyone feel about the Climate Change question last night? Exactly. #GOPDebate'
 "RT @ScottWalker: Didn't catch the full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…"
 'RT @TJMShow: No mention of Tamir Rice and the #GOPDebate was held in Cleveland? Wow.'
 ...
 'RT @Lrihendry: #TedCruz As President, I will always tell the truth, and do what I said I would do.  #GOPDebates'
 'RT @JRehling: #GOPDebate Donald Trump says that he doesn\'t have time for political correctness. How does calling women "fat pigs" save him …'
 'RT @Lrihendry: #TedCruz headed into the Presidential Debates. GO TED!! \n\n#GOPDebates http://t.co/8S67pz8a4A']
['Neutral' 'Positive' 'Neutral' ... 'Positive' 'Negative' 'Positive']


In [15]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [19]:

processed_features = []

for sentence in range(0, len(features)):  
  
    # Remove all the special characters
    processed_feature = re.sub(r'\W', ' ', str(features[sentence]))
    
    

  
    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    # Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Removing prefixed 'b'
    #processed_feature = re.sub(r'^b\s+', '', processed_feature)

  
    
    processed_feature = re.sub(r"i'm", "i am", processed_feature)
    processed_feature = re.sub(r"\'s", "is", processed_feature)
    #processed_feature = re.sub(r"she's", "she is", processed_feature)
    #processed_feature = re.sub(r"that's", "that is", processed_feature)
    #processed_feature = re.sub(r"what's", "what is", processed_feature)
    #processed_feature = re.sub(r"where's", "where is", processed_feature)
    #processed_feature = re.sub(r"how's", "how is", processed_feature)
    processed_feature = re.sub(r"\'ll", " will", processed_feature)
    processed_feature = re.sub(r"\'ve", " have", processed_feature)
    processed_feature = re.sub(r"\'re", " are", processed_feature)
    processed_feature = re.sub(r"\'d", " would", processed_feature)
    processed_feature = re.sub(r"n't", " not", processed_feature)
    processed_feature = re.sub(r"won't", "will not", processed_feature)
    processed_feature = re.sub(r"can't", "cannot", processed_feature)
    
    
    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    
    #Stemming Words
    from nltk.stem.porter import PorterStemmer
    stemmer = PorterStemmer()
    processed_feature=(' '.join([stemmer.stem(word) for word in processed_feature.split()]))
    #print (processed_feature)
    
    
    #Lemmatization
    
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    processed_feature=(' '.join([lemmatizer.lemmatize(word) for word in processed_feature.split()]))
    
    #stop_words = set(stopwords.words('english'))
    #processed_feature = (' '.join([word for word in processed_feature.split() if not word in stop_words]))

    # Converting to Lowercase
    processed_feature = processed_feature.lower()
    
    processed_features.append(processed_feature)
    
print (processed_features)


['rt nancyleegrahn how did everyon feel about the climat chang question last night exactli gopdeb', 'rt scottwalk didn catch the full gopdeb last night here are some of scott best line in 90 second walker16 http co zsff', 'rt tjmshow no mention of tamir rice and the gopdeb wa held in cleveland wow', 'rt robgeorg that carli fiorina is trend hour after her debat abov ani of the men in just complet gopdeb say she on', 'rt danscavino gopdeb realdonaldtrump deliv the highest rate in the histori of presidenti debat trump2016 http co', 'rt gregabbott_tx tedcruz on my first day will rescind everi illeg execut action taken by barack obama gopdeb foxnew', 'rt warriorwoman91 like her and wa happi when heard she wa go to be the moder not anymor gopdeb megynkelli http', 'go on msnbc live with thomasarobert around 2 pm et gopdeb', 'deer in the headlight rt lizzwinstead ben carson may be the onli brain surgeon who ha perform lobotomi on himself gopdeb', 'rt nancyosborne180 last night debat prove it g

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords 
stop_words = ['in', 'of', 'at', 'a', 'the','an','or','i','and','has','he','will','was','with','is','its','if']
#stop_words = set(stopwords.words('english'))
cv = CountVectorizer(binary=True,ngram_range=(1,2),stop_words=stop_words)
cv.fit(processed_features)
processed_features = cv.transform(processed_features)



In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)

In [22]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LogisticRegression
  
text_classifier_LR= LogisticRegression(C=0.5)
text_classifier_LR.fit(X_train,y_train)
#text_classifier_LR.fit(processed_features,labels)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
#print (type(X_test))
#print (X_test)
print ("Logist Regression Algorithm")
predictions = text_classifier_LR.predict(X_test.toarray()) 
from sklearn.metrics import classification_report, accuracy_score

print(accuracy_score(predictions,y_test))
print(classification_report(predictions,y_test))

Logist Regression Algorithm
0.7174774774774775
              precision    recall  f1-score   support

    Negative       0.90      0.74      0.81      2019
     Neutral       0.40      0.59      0.47       428
    Positive       0.51      0.72      0.60       328

    accuracy                           0.72      2775
   macro avg       0.60      0.69      0.63      2775
weighted avg       0.78      0.72      0.74      2775



In [0]:

sentence="Things are not going in a right way.Fix it as soon as possible";
vector_array = []
  
# Remove all the special characters
clean_input = re.sub(r'\W', ' ', str(sentence))




# Remove single characters from the start
clean_input = re.sub(r'\^[a-zA-Z]\s+', ' ', clean_input) 

# Substituting multiple spaces with single space
clean_input = re.sub(r'\s+', ' ', clean_input, flags=re.I)

# Removing prefixed 'b'
#processed_feature = re.sub(r'^b\s+', '', processed_feature)

# Converting to Lowercase
clean_input = clean_input.lower()

clean_input = re.sub(r"i'm", "i am", clean_input)
clean_input = re.sub(r"\'s", "is", clean_input)
#clean_input = re.sub(r"she's", "she is", clean_input)
#clean_input = re.sub(r"that's", "that is", clean_input)
#clean_input = re.sub(r"what's", "what is", clean_input)
#clean_input = re.sub(r"where's", "where is", clean_input)
#clean_input = re.sub(r"how's", "how is", clean_input)
clean_input = re.sub(r"\'ll", " will", clean_input)
clean_input = re.sub(r"\'ve", " have", clean_input)
clean_input = re.sub(r"\'re", " are", clean_input)
clean_input = re.sub(r"\'d", " would", clean_input)
clean_input = re.sub(r"n't", " not", clean_input)
clean_input = re.sub(r"won't", "will not", clean_input)
clean_input = re.sub(r"can't", "cannot", clean_input)

  # remove all single characters
clean_input= re.sub(r'\s+[a-zA-Z]\s+', ' ', clean_input)

vector_array.append(clean_input)
predict=cv.transform(vector_array)
#print (predict)





In [27]:
text_classifier_LR.predict(predict)

array(['Negative'], dtype=object)

In [28]:
frames_data=pd.read_excel('data.xlsx')
frames_data.head()

,Text,Response
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [29]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.linear_model import LogisticRegression
import random

#print( frames_data[frames_data['Response']==0]['Text'])
 
def word_feats(words):
    return dict([(word, True) for word in words])

#print(len(data_negative[data_negative['Response']=='Negative']['Negative_words']))

#labeled_names = ([(name, 'Negative') for name in data_negative[data_negative['Response']=='Negative']['Negative_words'] ] + [(name, 'Positive') for name in data_positive[data_positive['Response']=='Positive']['Postive_words'] ])
labeled_names = ([(str(name), 'Negative') for name in frames_data[frames_data['Response']==0]['Text'] ] + [(str(name), 'Positive') for name in frames_data[frames_data['Response']==1]['Text'] ])

random.shuffle(labeled_names)

print(len(labeled_names))

featuresets = [(word_feats(name[0].split(' ')),response) for name,response in labeled_names]

#print(featuresets[0])
#neg_featuresets = [(word_feats(name[0].split(' ')),'Negative') for name in labeled_names]
#print(neg_featuresets[0])
#pos_featuresets = [(word_feats(name[0].split(' ')),'Positive') for name in labeled_names]


print(len(labeled_names)*3/4)

#print(len(neg_featuresets))
#print(len(pos_featuresets))
 
#negcutoff = int(len(neg_featuresets)*3/4)
#print(negcutoff)
#poscutoff = int(len(pos_featuresets)*3/4)
#print(poscutoff)

#train_set = negfeats[:negcutoff] + posfeats[:poscutoff]



#test_set = negfeats[negcutoff:] + posfeats[poscutoff:]

train_set, test_set = featuresets[:2061], featuresets[2061:]

#print(train_set[0])

#train_set = (labeled_names[:8046])
#print (trainfeats[0])
#test_set= (labeled_names[8046:] )
print(train_set[0])
print ('train on {} instances, test on {} instances'.format(len(train_set), len(test_set)))

LogisticRegression_classifier = SklearnClassifier(LogisticRegression(C=0.01))
classifier=LogisticRegression_classifier.train(train_set)
print ('accuracy:', nltk.classify.util.accuracy(classifier, test_set))
save_classifier = open("naivebayes.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()
#classifier_f = open("naivebayes.pickle", "rb")
#classifier = pickle.load(classifier_f)
#classifier_f.close()
#classifier.show_most_informative_features()

2748
2061.0
({'D': True}, 'Negative')
train on 2061 instances, test on 687 instances
accuracy: 0.5240174672489083


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [0]:
n = data.columns[15]

# Drop that column
data.drop(n, axis = 1, inplace = True)

# Put whatever series you want in its place
data[n] =processed_features 

In [31]:
test=" Good enough to pass "
test_features=word_feats(test)
print(test_features)

a=classifier.prob_classify(test_features)
for label in a.samples():
    print("%s: %f" % (label, a.prob(label)))

{' ': True, 'G': True, 'o': True, 'd': True, 'e': True, 'n': True, 'u': True, 'g': True, 'h': True, 't': True, 'p': True, 'a': True, 's': True}
Negative: 0.444359
Positive: 0.555641
